# Setting up
Importing, loading data from proteomeHD csv

Importing libraries

In [1]:
import networkx as nx
from networkx.algorithms.community import k_clique_communities
from networkx.algorithms.clique import find_cliques
import pandas as pd
import numpy as np
import time
import math
import multiprocessing as mp
from numba import jit
from csv import writer
import glob
import matplotlib.pyplot as plt

Declaring global variables of
1. proteomeHD_csv_path: Path to the csv file containing data used in the proteomeHD paper

In [2]:
directory_path = "D:\\Desktop\\Northeastern_University\\Research\\Proteomics\\ProteinProteinAssociation\\Development\\"
proteomeHD_csv_path = f"{directory_path}data_sources\\ProteomeHD\\ProteomeHD_v1_1.csv"

Taking a look at the data
1. There are 10323 rows and 298 columns in total
2. The first four columns are properties of proteins
3. Each of the last 294 columns represents an experiment
4. The numbers represent the quantities of protein after each experiment

In [3]:
proteomHD_dataframe = pd.read_csv(proteomeHD_csv_path) 
total_rows = len(proteomHD_dataframe.index)
proteomHD_dataframe.head()

,Majority_protein_IDs,Simplified_protein_ID,Protein_names,Gene_names,RatioHL_GK1_Chromatin_AL,RatioHL_GK1_Chromatin_CPT,RatioHL_GK1_Chromatin_CR,RatioHL_GK1_Chromatin_HepHek,RatioHL_GK1_Chromatin_hiIR,RatioHL_GK1_Chromatin_loIR,...,RatioHL_PX441_E1,RatioHL_PX441_E2,RatioHL_PX441_E3,RatioHL_PX441_E4,RatioHL_PX441_E5,RatioHL_PX441_F1,RatioHL_PX441_F2,RatioHL_PX441_F3,RatioHL_PX441_F4,RatioHL_PX441_F5
0,A0AV96-2;A0AV96,A0AV96,RNA-binding protein 47,RBM47,-0.274172,NaN,-0.52595,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0AVF1-2;A0AVF1-3;A0AVF1,A0AVF1,Tetratricopeptide repeat protein 26,TTC26,NaN,NaN,NaN,NaN,NaN,0.273198,...,NaN,NaN,NaN,NaN,NaN,-3.108931,NaN,NaN,NaN,NaN
2,A0AVI4-2;A0AVI4,A0AVI4,E3 ubiquitin-protein ligase TM129,TMEM129,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0AVT1;A0AVT1-2,A0AVT1,Ubiquitin-like modifier-activating enzyme 6,UBA6,-0.493533,0.66084,-0.22916,1.525756,0.322934,0.452473,...,0.373395,0.335653,0.912479,-0.346645,NaN,0.368652,-0.105655,-0.233525,-0.447211,0.283507
4,A0FGR8-2;A0FGR8;A0FGR8-4,A0FGR8,Extended synaptotagmin-2,ESYT2,-0.997325,NaN,NaN,1.019208,NaN,NaN,...,-0.129918,-0.203415,-0.433365,-0.238946,-0.850058,-0.378592,-0.057059,0.123395,-0.336181,0.478821


In [4]:
proteomeHD_simplified_protein_column = proteomHD_dataframe['Simplified_protein_ID'].to_numpy()
proteomeHD_majority_protein_column = proteomHD_dataframe['Majority_protein_IDs'].to_numpy()

# Defining functions
Defining functions that will help calculate interaction between proteins

Defining helper functions for data retrieval
1. get_feature_vector: Returns a 1 x 294 numpy vector representing experimental data where NaN are replaced by 0
2. get_protein_attributes: Returns information about the protein located at the given row




In [5]:
def get_feature_vector(row_number):
  return proteomHD_dataframe.iloc[int(row_number),4:-1].fillna(0).to_numpy()

def get_protein_attributes(row_number):
  return proteomHD_dataframe.iloc[int(row_number),0:4]

Defining functions that helps with r2 calculations:
1. find_common_observations: Given two protein feature vectors, returns the perturbation data points of two proteins where they both have data (non-zero)
2. find_associations: Given row number of a protein, a r2 cutoff, an observation threshold, return a list that contains the tuple (protein1, protein2, r2). Where protein1 is the row number of the initial given protein, protein2 is the row number of a protein that satisfy the following requirements: have non-zero data points with protein1 in perturbations more than the specified observation threshold, and the r2 value of these data points are more than the specified r2 cutoff. The r2 is the r2 value between these two proteins.

In [6]:
# find_common_observations: 
# Given vector 1 and 2, return subsets x1 of vector 1 x2 of vector2 where the
# original vector values are both nonzero and return the indices 
def find_common_observations(vector1,vector2):
  vector1_bool = np.where(vector1 != 0, 1, 0)
  vector2_bool = np.where(vector2 != 0, 1, 0)
  take_indices = np.logical_and(vector1_bool,vector2_bool)
  take_indices = take_indices.nonzero()[0]
  x1 = np.take(vector1,take_indices)
  x2 = np.take(vector2,take_indices)
  return x1,x2

# find_associations: Returns an array of tuples, tuples contain (majority protein ids, gene names, r2)
# protein: row number of protein
# r2_threshold: the minimum r2 value it takes to associate with other proteins
# observation_threshold: the minimum amount of co-observations it takes to associate with other proteins
# look_below_only: If true, only compare the current protein against proteins below the row number of the
# the current protein. Recommended to be True during large batch of computations since r2 is symmetric
def find_associations(protein,r2_threshold=0.9,observation_threshold=6,look_below_only=True):
  protein_vec = get_feature_vector(protein)
  r2_vector = np.zeros(total_rows)
  # wrapper for jit (speeds up computation)
  find_common_observations_numba = jit()(find_common_observations)
  start_row = protein if look_below_only else 0
  for index in range(start_row,total_rows):
    vec_to_compare = get_feature_vector(index)
    x1,x2 = find_common_observations_numba(protein_vec,vec_to_compare)
    if(len(x1) >= observation_threshold):
      r2 = np.corrcoef(x1,x2)[0,1]**2
      if r2 >= r2_threshold:
        r2_vector.itemset(index,r2)
  return protein,r2_vector

In [7]:
# start = time.time()
# for i in range(50000):
#   r2 = np.corrcoef(np.random.rand(10),np.random.rand(10))[0,1]**2
# print(time.time()-start)
# #np.corrcoef(np.rand(10))

In [8]:
# def find_common_observations_num(vector1,vector2):
#   vector1_bool = np.where(vector1 != 0, 1, 0)
#   vector2_bool = np.where(vector2 != 0, 1, 0)
#   take_indices = np.logical_and(vector1_bool,vector2_bool)
#   return np.count_nonzero(take_indices)
# start_time = time.time()
# co_observation_matrix = np.zeros((100,100))
# for i in range(100):
#   for j in range(i,100):
#     num = find_common_observations_num(get_feature_vector(i),get_feature_vector(j))
#     co_observation_matrix.itemset((i,j),num)

# print(time.time()-start_time)
# print(co_observation_matrix)

Defining helper functions and variables that generates csv
1. row_start: from which row do we start computing protein interactions
2. row_end: from which row do we end computing protein interactions
3. interactions_tuple_file_write_path: Path to file where the protein-protein interaction tuples csv will be stored


In [0]:
row_start = 10000
row_end = 10323
interactions_tuple_file_write_path = '/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/csv_outputs/ppa_{row_start}_{row_end}.csv'.format(row_start=row_start,row_end=row_end)

Defining helper functions that writes to csv
1. append_to_csv: appends given vectors to csv
2. find_associations_wrapper: wrapper of the function find_associations, written so we can make use of the multiprocessing library

In [0]:
# append_to_csv: appends every element in the given list, each being a row, to the csv
# file_name: name of csv to be appended to
# elems: list of tuples
def append_to_csv(file_name, protein, r2_vector):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        indices = r2_vector.nonzero()[0]
        for index in indices:
          csv_writer.writerow((protein,index,r2_vector[index]))

# find_associations_wrapper: Wrapper function of the function find_associations, to be used for multiprocessing library
# protein: the protein of interest, we will write all interactions of this protein to the specified csv (interactions_tuple_file_write_path)
def find_associations_wrapper(protein):
  print("Current Protein:")
  print(protein)
  protein,r2_vec = find_associations(protein)
  append_to_csv(interactions_tuple_file_write_path,protein,r2_vec)

# batch_run: Generates the protein-protein interaction tuples in the given range and write them to csv (interactions_tuple_file_write_path)
def batch_run(start,end):
  batch = range(start,end)
  processors=mp.Pool(8)
  start_time = time.time()
  processors.map(find_associations_wrapper,batch)
  print('Done processing the batch from row {start} to row {end}'.format(start=start,end=end))
  print()
  print("Time Elapsed:")
  print(time.time() - start_time)

# Perform Batch Protein Interaction Computations
Actually writing the interactions to csv

**Warning!** Running the following block uncommented will start the batch run. row_start and row_end are defined in the previous section

In [0]:
# batch_run(row_start,row_end)

# Convert CSV to NetworkX graph
So far these are the code used to generate protein-protein interaction tuples and write them to csv. The next part is about reading these csv and constructing networkx graphs.

In [16]:
all_proteomeHD_pairs_path = f"{directory_path}data_sources\\ProteomeHD\\all_proteomeHD_pairs.csv"
all_proteomeHD_pairs_df = pd.read_csv(all_proteomeHD_pairs_path)

In [18]:
all_proteomeHD_pairs_df.describe()

,r,r2,observations
count,5.326798e+07,5.326798e+07,5.327700e+07
mean,5.042782e-03,1.669958e-01,4.830352e+01
std,4.086200e-01,2.608977e-01,5.887994e+01
min,-1.000000e+00,0.000000e+00,0.000000e+00
25%,-2.183153e-01,5.549306e-03,6.000000e+00
50%,0.000000e+00,4.940056e-02,2.200000e+01
75%,2.263308e-01,1.963053e-01,7.200000e+01
max,1.000000e+00,1.000000e+00,2.940000e+02


Filter down further, if needed

In [19]:
filtered_df = all_proteomeHD_pairs_df[all_proteomeHD_pairs_df['r'] >= 0.75]
filtered_df = filtered_df[filtered_df['observations'] >= 100]

In [20]:
filtered_df

,protein1_majority_name,protein2_majority_name,protein1_simplified_name,protein2_simplified_name,r,r2,observations
36638,A0AVT1;A0AVT1-2,Q70IA6;Q70IA6-3;Q70IA6-2,A0AVT1,Q70IA6,0.762766,0.581812,107
37694,A0AVT1;A0AVT1-2,Q8TDQ7;Q8TDQ7-2;Q8TDQ7-5;Q8TDQ7-3,A0AVT1,Q8TDQ7,0.776390,0.602782,113
42964,A0FGR8-2;A0FGR8;A0FGR8-4,P08567,A0FGR8,P08567,0.777896,0.605123,109
45398,A0FGR8-2;A0FGR8;A0FGR8-4,Q13094,A0FGR8,Q13094,0.819053,0.670847,100
48517,A0FGR8-2;A0FGR8;A0FGR8-4,Q96BY7,A0FGR8,Q96BY7,0.799563,0.639300,113
...,...,...,...,...,...,...,...
53243302,Q9Y3B8;Q9Y3B8-2,Q9Y6M1-1;Q9Y6M1;Q9Y6M1-5;Q9Y6M1-3;Q9Y6M1-4,Q9Y3B8,Q9Y6M1,0.770505,0.593679,102
53246751,Q9Y3D8;Q9Y3D8-2,Q9Y5Q8-3,Q9Y3D8,Q9Y5Q8,0.816931,0.667376,147
53255602,Q9Y490,Q9Y624;Q9Y624-2,Q9Y490,Q9Y624,0.801182,0.641893,183
53266362,Q9Y584,Q9Y6C9,Q9Y584,Q9Y6C9,0.782675,0.612580,125


Define variables, initial graph and helper functions
1. interactions_tuple_directory_read_path: Path to directory that contains all the csv that has protein-protein interaction tuples

In [36]:
import time
# interactions_tuple_directory_read_path = "/content/drive/My Drive/Colab Notebooks/Research/ProteinProteinAssociation/csv_outputs/ppa_r2_90_co_5/*.csv"
ppa_graph = nx.Graph()

# Initializes the graph by adding protein as nodes along with their attributes
def initialize_ppa_graph():
  ppa_graph.clear()
  for i in range(total_rows):
    ppa_graph.add_node(i,Majority_protein_IDs=proteomeHD_majority_protein_column[i],Simplified_protein_ID=proteomeHD_simplified_protein_column[i])

# # Adds a r2 relation between proteins
# def ppa_add_relation(protein1,protein2,r,r2,observations):
#   ppa_graph.add_edge(protein1,protein2,r=r,r2=r2,observations=observations)

# Process a single csv by adding r2 relation in the csv to the network (skipping self to self interactions)
def process_single_df(df):
  start_time = time.time()
  for idx,row in df.iterrows():
    protein1 = np.where(proteomeHD_majority_protein_column == row['protein1_majority_name'])[0][0]
    protein2 = np.where(proteomeHD_majority_protein_column == row['protein2_majority_name'])[0][0]
    r = float(row['r'])
    r2 = float(row['r2'])
    observations = int(row['observations'])
    ppa_graph.add_edge(protein1,protein2,r=r,r2=r2,observations=observations)
    if idx % 10000 == 0:
      percent_done = (idx+1) / len(df)
      print(f"Percent done: {percent_done}")
      time_since_start = time.time() - start_time
      print(f"Time elasped: {time_since_start}")
      print(f"Estimate finishing in: {time_since_start / (percent_done) - time_since_start}")

# # Process all csvs in the given path and load them to the network
# def process_r2_csvs():
#   for fname in glob.glob(interactions_tuple_directory_read_path):
#     process_single_csv(fname)
#   print("All file processed")

Initializes and loads the network

In [37]:
initialize_ppa_graph()
process_single_df(filtered_df)

Percent done: 915.2479806138933
Time elasped: 1.9546451568603516
Estimate finishing in: -1.9525095114415556


Check network data

In [38]:
print("Number of nodes")
print(ppa_graph.number_of_nodes())
print("Number of edges")
print(ppa_graph.number_of_edges())

Number of nodes
10323
Number of edges
16094


In [40]:
ppa_graph.nodes[0]['Simplified_protein_ID']

'A0AV96'

# Save graph
Saving the graph as a .graphml file


networkx_write_path: Path to export networkx graph after the network is constructed

In [41]:
#outputs network data as graphml
networkx_write_path = f"{directory_path}\\graph_outputs\\ppa_r_075_obs_100.graphml"
nx.write_graphml(ppa_graph, networkx_write_path)